# AWS S3 with Boto3 

Jenny Smith

June 24, 2020

In [1]:
import os
import re
import itertools
import boto3
import logging
from botocore.exceptions import ClientError
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
os. getcwd()
os.chdir("/Users/work/home_drive/scripts/STAR-fusion-NF/sample_sheets/")
os. getcwd()

'/Users/work/home_drive/scripts/python'

'/Users/work/home_drive/scripts/STAR-fusion-NF/sample_sheets'

# 1. Read in the Manifest file

In [3]:
manifest = pd.read_csv("/Users/work/fast_drive/workingDir/TARGET/AML_TARGET/SequencingDataMatrix/TARGET_AML_Ribodepleted_Master_Manifest_5.29.20.csv", 
                       engine='python')

manifest.head()
manifest.shape

,Sample,USI,AML_Subtype,Group,Batch,Library,Time_point,NUP98.NSD1,DEK.NUP214,FUS.ERG,...,CNS.disease,Chloroma,ETS.CNVs,rs12459419,Comments,RBM15.MKL1,NUP98.KDM5A,NUP98.Rearranged,NUP98.Rearranged.Groups,CNS.Disease.Harmonized
0,Kasumi.AZA.D11.03A.01R,Kasumi,CellLine,CellLine,dx1,A75511,CellLine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CellLine,NaN
1,Kasumi.AZA.D5.03A.01R,Kasumi,CellLine,CellLine,dx1,A75509,CellLine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CellLine,NaN
2,Kasumi.D1.03A.01R,Kasumi,CellLine,CellLine,dx1,A75507,CellLine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CellLine,NaN
3,MV4.11.AZA.D11.03A.01R,MV4,CellLine,CellLine,dx1,A75512,CellLine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CellLine,NaN
4,MV4.11.AZA.D5.03A.01R,MV4,CellLine,CellLine,dx1,A75510,CellLine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CellLine,NaN


(2345, 155)

In [4]:
#table for the different Batchs 
manifest.Batch.value_counts()

dx1      1117
dx2       457
rlps2     302
ds1       228
rlps1     198
rlps3      36
rlps4       7
Name: Batch, dtype: int64

# 2. Subset for Required Samples

In [5]:
ds_aml = manifest[manifest.Batch.str.contains("ds")]

ds_aml.head()
ds_aml.shape #228 rows

,Sample,USI,AML_Subtype,Group,Batch,Library,Time_point,NUP98.NSD1,DEK.NUP214,FUS.ERG,...,CNS.disease,Chloroma,ETS.CNVs,rs12459419,Comments,RBM15.MKL1,NUP98.KDM5A,NUP98.Rearranged,NUP98.Rearranged.Groups,CNS.Disease.Harmonized
2117,PAXUMA.03A.01R,PAXUMA,DS,DS,ds1,B35245,diagnostic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DS,NaN
2118,PAXUTK.09A.01R,PAXUTK,DS,DS,ds1,B35246,diagnostic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DS,NaN
2119,PAXVDP.03A.01R,PAXVDP,DS,DS,ds1,B35247,diagnostic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DS,NaN
2120,PAXVFA.09A.01R,PAXVFA,DS,DS,ds1,B35248,diagnostic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DS,NaN
2121,PAVZTK.09A.01R,PAVZTK,DS,DS,ds1,B35249,diagnostic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DS,NaN


(228, 155)

# 3. Run the Sample Sheet Script